In [1]:
import openai
import pandas as pd
import numpy as np
import pickle
# from transformers import GPT2TokenizerFast
from typing import List
import json

In [2]:
# Loading OpenAI Key
openai.api_key = "sk-9******************"

In [3]:
# ChatGPT Model
COMPLETIONS_MODEL = "text-davinci-003"

## model hallucination

In [4]:
prompt = "When babar scored fourth t20I hundred?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")



'Babar Azam scored his fourth T20I hundred on October 24, 2020, against Zimbabwe in Rawalpindi.'

In [5]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: how many hundreds does babar azam scored in T20I?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

In [6]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Gianmarco Tamberi and Mutaz Essa Barshim emerged as joint winners of the event.'

## Data Loading

In [7]:
df = pd.read_csv('https://cdn.openai.com/API/examples/data/olympics_sections_text.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

3964 rows in the data.


,,content,tokens
title,heading,,
Gymnastics at the 2020 Summer Olympics – Men's artistic team all-around,Qualified teams,"To reach the Olympics, a National Olympic Comm...",167
South Korea at the 2020 Summer Olympics,Women's tournament,South Korea women's basketball team qualified ...,74
Concerns and controversies at the 2020 Summer Olympics,Raven Saunders' hand gesture,After receiving her silver medal during the me...,158
Qatar at the 2020 Summer Olympics,Judo,Qatar qualified one judoka for the men's half-...,69
Mexico at the 2020 Summer Olympics,Sailing,Mexican sailors qualified one boat in each of ...,47


In [8]:
print(df.loc[df.index[10], "content"])

In February 2012, it was announced that Tokyo's former National Stadium, the central venue for the 1964 Summer Olympics, would undergo a ¥100 billion renovation for the 2019 Rugby World Cup and the 2020 Summer Olympics. In November 2012, the Japan Sport Council announced that it was taking bids for proposed stadium designs. Of the 46 finalists, Zaha Hadid Architects was awarded the project, which would replace the old stadium with a new 80,000-seat stadium. There was criticism of the Zaha Hadid design—which was compared to a bicycle helmet and regarded as clashing with the surrounding Meiji Shrine—and widespread disapproval of the costs, even with attempts to revise and "optimize" the design.In June 2015, the government announced plans to reduce the new stadium's permanent capacity to 65,000 in its athletics configuration (although with the option to add up to 15,000 temporary seats for football) as a further cost-saving measure. The original plan to build a retractable roof was also a

# Text Embedding

In [9]:
def get_embedding(text):
  # Note how this function assumes you already set your Open AI key!
    result = openai.Embedding.create(
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]

In [10]:
get_embedding(df['content'][1])

[0.009944994933903217,
 -0.012711109593510628,
 -0.010667438618838787,
 -0.0048976498655974865,
 -0.02543523535132408,
 0.019499478861689568,
 -0.03553643450140953,
 -0.007640984375029802,
 -0.007959901355206966,
 -0.023326480761170387,
 0.021321861073374748,
 -0.00027884874725714326,
 0.003413710743188858,
 -0.027543991804122925,
 -0.025226963683962822,
 -0.0019444157369434834,
 0.017091333866119385,
 -0.0022747223265469074,
 0.003644762560725212,
 0.0020420432556420565,
 -0.02184254117310047,
 0.008051020093262196,
 -0.003220082726329565,
 0.0011568866902962327,
 0.007341593038290739,
 -0.007048710249364376,
 0.0218815915286541,
 -0.014396811835467815,
 0.011038423515856266,
 -0.014930509962141514,
 -0.005958536174148321,
 0.020410669967532158,
 -0.019382325932383537,
 -0.002945098327472806,
 -0.021178673952817917,
 -0.00016413634875789285,
 -0.017208486795425415,
 -0.017768217250704765,
 -0.00530768558382988,
 -0.01046567503362894,
 -0.002115264069288969,
 0.01728658750653267,
 -0.0

In [11]:
#Get Embeddings of All the text in Content Column

df['embedding'] = df['content'].apply(get_embedding)

In [12]:
df.head()

content  \
title                heading                                                                                             
2020 Summer Olympics Summary                                         The 2020 Summer Olympics (Japanese: 2020年夏季オリン...   
                     Host city selection                             The International Olympic Committee (IOC) vote...   
                     Impact of the COVID-19 pandemic                 In January 2020, concerns were raised about th...   
                     Qualifying event cancellation and postponement  Concerns about the pandemic began to affect qu...   
                     Effect on doping tests                          Mandatory doping tests were being severely res...   

                                                                     tokens  \
title                heading                                                  
2020 Summer Olympics Summary                                            726   
                     Host city selection                                126   
                     Impact of the COVID-19 pandemic                    374   
                     Qualifying event cancellation and postponement     298   
                     Effect on doping tests                             163   

                                                                                                             embedding  
title                heading                                                                                            
2020 Summer Olympics Summary                                         [0.0037261785473674536, -0.006241673603653908,...  
                     Host city selection                             [0.009944994933903217, -0.012711109593510628, ...  
                     Impact of the COVID-19 pandemic                 [-0.0032951163593679667, -0.02189432829618454,...  
                     Qualifying event cancellation and postponement  [-0.008610822260379791, -0.03721671178936958, ...  
                     Effect on doping tests                          [-0.006291721481829882, -0.022288408130407333,...

In [15]:
df.to_csv('olympics-data/olympics_unicorns_with_embeddings.csv',index=False)

In [4]:
df = pd.read_csv('Embedding datasets/olympics_unicorns_with_embeddings.csv')

In [5]:
df

,content,tokens,embedding
0,The 2020 Summer Olympics (Japanese: 2020年夏季オリン...,726,"[0.0037191431038081646, -0.0062797050923109055..."
1,The International Olympic Committee (IOC) vote...,126,"[0.009944994933903217, -0.012711109593510628, ..."
2,"In January 2020, concerns were raised about th...",374,"[-0.0032951163593679667, -0.02189432829618454,..."
3,Concerns about the pandemic began to affect qu...,298,"[-0.008610822260379791, -0.03721671178936958, ..."
4,Mandatory doping tests were being severely res...,163,"[-0.006291721481829882, -0.022288408130407333,..."
...,...,...,...
3959,Haiti competed at the 2020 Summer Olympics in ...,106,"[-0.020807543769478798, -0.0003147805691696703..."
3960,Haiti received a universality slot from the Wo...,42,"[-0.009769239462912083, 0.010949461720883846, ..."
3961,Haiti qualified one judoka for the women's hal...,67,"[-0.0017669345252215862, 0.01737343892455101, ..."
3962,Haiti received a universality invitation from ...,49,"[0.0018741308012977242, 0.0037449179217219353,..."


## Document Similarity

In [13]:
#For the similarity function for the second time
df['embedding'] = df['embedding'].apply(json.loads) 

In [6]:
# prompt = "when Chinese anti-doping agency temporarily ceased testing?"
prompt ="when the Japan Sport Council announced that it was taking bids for proposed stadium designs"

In [7]:
prompt_embedding = get_embedding(prompt)

NameError: name 'get_embedding' is not defined

In [16]:
prompt_embedding

[0.0004656231321860105,
 -0.002875307807698846,
 0.0016398770967498422,
 0.01553889736533165,
 -0.009332854300737381,
 0.01708870753645897,
 -0.036325402557849884,
 -0.021466245874762535,
 0.010930247604846954,
 -0.015647655352950096,
 0.02925608679652214,
 -0.002066415036097169,
 0.02153421938419342,
 -0.024742601439356804,
 -0.020337874069809914,
 -0.017618907615542412,
 0.013268559239804745,
 0.010984627529978752,
 -0.008591935969889164,
 -0.005927348509430885,
 -0.0009414423839189112,
 0.012758753262460232,
 -0.005461725406348705,
 -0.009917432442307472,
 -0.0023366124369204044,
 0.008986186236143112,
 0.025884566828608513,
 -0.026156464591622353,
 -0.010264100506901741,
 0.02135748602449894,
 -0.00033923363662324846,
 0.009625143371522427,
 -0.013975490815937519,
 -0.017455769702792168,
 -0.01644975133240223,
 -0.022771349176764488,
 -0.002532038139179349,
 -0.0006890882505103946,
 0.008523961529135704,
 -0.0058559756726026535,
 0.02143905498087406,
 0.0034717812668532133,
 0.0002

In [17]:
# Function to calculate the Similarity for all the data
import numpy as np

# Assuming 'embedding' column contains list representations of vectors

def vector_similarity(vec1, vec2):
    """
    Returns the similarity between two vectors.
    """
    return np.dot(np.array(vec1), np.array(vec2))



In [51]:
# df.to_csv('olympics-data/olympics_unicorns_with_embeddings+Similarity.csv',index=False)

In [4]:
# df = pd.read_csv('olympics-data/olympics_unicorns_with_embeddings+Similarity.csv')

In [18]:
# prompt_embedding = get_embedding(prompt)  # Assuming this returns a list

df['prompt_similarity'] = df['embedding'].apply(lambda vector: vector_similarity(vector, prompt_embedding))


In [19]:
df["prompt_similarity"]

0       0.803948
1       0.829631
2       0.805313
3       0.787394
4       0.764333
          ...   
3959    0.772736
3960    0.763709
3961    0.763164
3962    0.780545
3963    0.756646
Name: prompt_similarity, Length: 3964, dtype: float64

In [21]:
# Finding most similar Context to a specific prompt
df.nlargest(1,'prompt_similarity')

,content,tokens,embedding,prompt_similarity
1468,"In February 2012, it was announced that former...",415,"[0.004500451032072306, 0.0008055070065893233, ...",0.895284


In [5]:
df

,content,tokens,embedding,prompt_similarity
0,The 2020 Summer Olympics (Japanese: 2020年夏季オリン...,726,"[0.003723830683156848, -0.006204220000654459, ...",0.767209
1,The International Olympic Committee (IOC) vote...,126,"[0.009945299476385117, -0.012757059186697006, ...",0.788214
2,"In January 2020, concerns were raised about th...",374,"[-0.0032519560772925615, -0.021974382922053337...",0.782543
3,Concerns about the pandemic began to affect qu...,298,"[-0.008643084205687046, -0.03723987564444542, ...",0.778086
4,Mandatory doping tests were being severely res...,163,"[-0.006218883208930492, -0.022525297477841377,...",0.764455
...,...,...,...,...
3959,Haiti competed at the 2020 Summer Olympics in ...,106,"[-0.02083267644047737, -0.00023612691438756883...",0.746992
3960,Haiti received a universality slot from the Wo...,42,"[-0.009835230186581612, 0.010927274823188782, ...",0.745175
3961,Haiti qualified one judoka for the women's hal...,67,"[-0.0014589326456189156, 0.01703108288347721, ...",0.740813
3962,Haiti received a universality invitation from ...,49,"[0.001765138586051762, 0.0037542625796049833, ...",0.753030


In [22]:
context = df.nlargest(1,'prompt_similarity').iloc[0]['content']

In [23]:
prompt = f'''Only answer the question below if you have 100% certainty of the facts.
Here is some Context: {context}
Q:when the Japan Sport Council announced that it was taking bids for proposed stadium designs?
A:'''

In [24]:
print(prompt)

Only answer the question below if you have 100% certainty of the facts.
Here is some Context: In February 2012, it was announced that former Tokyo's National Stadium, the central venue for the 1964 Summer Olympics, would undergo a ¥100 billion renovation for the 2019 Rugby World Cup and the 2020 Summer Olympics. In November 2012, the Japan Sport Council announced it was taking bids for proposed stadium designs. Of the 46 finalists, Zaha Hadid Architects was awarded the project, which would replace the old stadium with a new 80,000-seat stadium. There was criticism of the Zaha Hadid design—which was compared to a bicycle helmet and regarded as clashing with the surrounding Meiji Shrine—and widespread disapproval of the costs, even with attempts to revise and "optimize" the design.In June 2015, the government announced it was planning to reduce the new stadium's permanent capacity to 65,000 in its athletics configuration (although with the option to add up to 15,000 temporary seats for f

In [25]:
response = openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=500,
    model="text-davinci-003"
)

In [26]:
print(response['choices'][0]['text'])

 November 2012


In [27]:
def embed_prompt_lookup():
    # initial question
    question = input("What question do you have about a Olympics 2020?\n ")
    # Get embedding
    prompt_embedding = get_embedding(question)
    # Get prompt similarity with embeddings
    # Note how this will overwrite the prompt similarity column each time!
    df["prompt_similarity"] = df['embedding'].apply(lambda vector: vector_similarity(vector, prompt_embedding))

    # get most similar summary
    context = df.nlargest(1,'prompt_similarity').iloc[0]['content'] 

    prompt = f"""Only answer the question below if you have 100% certainty of the facts, use the context below to answer.
            Here is some context:
            {context}
            Q: {question}
            A:"""


    
    response = openai.Completion.create(
        prompt=prompt,
        temperature=0,
        max_tokens=500,
        model="text-davinci-003"
    )
    print(response["choices"][0]["text"].strip(" \n"))

In [28]:
embed_prompt_lookup()

What question do you have about a Olympics 2020?
  when the Japan Sport Council announced that it was taking bids for proposed stadium designs


November 2012
